# **End Phase 2 Project**

In [ ]:
# importing necessary libraries

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
import statistics

import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split

from scipy.stats import ttest_ind, ttest_1samp, chi2_contingency, norm

from sklearn.preprocessing import StandardScaler, LabelEncoder

import os
import pickle

import warnings
warnings.filterwarnings('ignore')